In [19]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import math
from Channel_Generation_5 import Channel_Generation_5 as GC


In [20]:
# Deciding the system parameters
# Tx Bandwidth is 100 MHz

K = 3     # No. of users
noise = -170 + 10 * np.log10(1000 * 1e6)   # noise in dBm
print(noise)
ite = 1000                 #no of channel realisations
noise = 10 ** (noise/10 - 3) #linear scale
print(noise)
N = 150                      #no of elements in IRS
M = 4

-80.0
1e-11


In [21]:
power_dBm = np.array(range(5, 21, 2))
P_mW = 10 ** (power_dBm/10) * 0.001
P_range = len(P_mW)


# Initialize the beamforming matrix W randomly
W_1 =  np.random.rand(M,K) + 1j * np.random.rand(M,K)
W = np.zeros((M,K), dtype='complex')
W_ni = np.zeros((M,K), dtype='complex')
#for p in range(P_range):
for i in range(K):
  W[:,i] = np.sqrt(1/K) * W_1[:,i] / np.linalg.norm(W_1[:,i])
  W_ni[:,i] = W_1[:,i] / np.linalg.norm(W_1[:,i])


# Initialising the phases at IRS
phase0 = np.random.rand(N)*2*np.pi
v = np.exp(1j*phase0)
Theta = np.diag(v)

In [22]:
Hr= np.zeros((P_range, ite, K, N), dtype= complex)
Hd= np.zeros((P_range, ite, K, M), dtype= complex)
G= np.zeros((P_range, ite, N, M), dtype= complex)
for p in range(P_range):
  for j in range(ite):
    CG= GC(K, M, N)
    Hd2, Hr2, G2= CG.Generate_Channels()
    Hd[p, j, :, :]= Hd2.T
    Hr[p, j, :, :]= Hr2.T
    G[p, j, :, :]= G2

print('Hd=',Hd[0,0,:,:].shape)
print('Hr=',Hr[0,0,:,:].shape)
print('G=',G[0,0,:,:].shape)

Hd= (3, 4)
Hr= (3, 150)
G= (150, 4)


ALTERNATING OPTIMISATION

In [26]:
Ch_eq = np.zeros((P_range,ite,K,M), dtype='complex')
P_user = np.zeros((P_range,K))
print(Ch_eq.shape)
print(W.shape)

(8, 1000, 3, 4)
(4, 3)


In [24]:

def Alternating_Optimisation( P, K):
    iter_max = 4


    theta_AO = np.zeros((P_range, ite,N,K))
    rate_AO = np.zeros((K))
    rate_i = np.zeros((ite,K, iter_max))
    print(P)
    for j in range(ite):
        W = np.sqrt(P/K) * W_1 / np.linalg.norm(W_1,axis=0)
        
        for i in range(K):
            temp = 0
            for iter in range(iter_max):
                theta= np.angle(Hd[j,i,:] @ W[:,i]) - np.angle((np.diag(Hr[j,i,:]) @ G[j,:,:] @ W[:,i])) 
                v = np.exp(1j*theta)
                Theta = np.diag(v)  

                Ch_eq[j,i,:] = Hd[j,i,:] + Hr[j,i,:] @ Theta @ G[j,:,:]
                W[:,i] = np.sqrt(P/K) * np.conj(Ch_eq[j,i,:]).T / np.linalg.norm(Ch_eq[j,i,:])  #MRT equation
            for k in range(K):
                if k != i:
                    temp += (np.linalg.norm(Ch_eq[j,i,:] @ W[:,k]))**2

            rate_i[j,i,iter] = np.log2(1+(np.linalg.norm(Ch_eq[j,i,:] @ W[:,i]) **2)/ (temp + noise))
                #print(rate_i[j,iter])
            theta_AO[j,:,i] = theta  # optimised Phase Shifts value for user 'x'
    for i in range(K):
        rate_AO[i] = np.mean(rate_i[:,i,iter_max-1], axis=0)
    
    return  theta_AO
    
     

need to fix the phase-shift for 3 users, taking one at a time

In [25]:
for p in range(P_range):
    P = 10 ** (power_dBm[p]/10) * 0.001
    
    theta0_zf = Alternating_Optimisation(P, K)
    print(theta0_zf)



0.0031622776601683794


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 150 is different from 3)